<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Load ClearScape from Google Storage Bucket
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

In [1]:
%connect local

Password: ·············


Success: 'local' connection established and activated for user 'demo_user', with default database 'demo_user'


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>2. Creat Hospital_Revenue_Factors table from S3 Buckets</b></p>


In [ ]:
SELECT
  TOP 10 *
FROM (
	LOCATION='/gs/storage.googleapis.com/solomon_demo_data/CSVDATA/Hospital_Revenue_Factors.parquet'
) AS d;

In [ ]:
CREATE FOREIGN TABLE
Hospital_Revenue_Factors_F
USING ( LOCATION('/gs/storage.googleapis.com/solomon_demo_data/CSVDATA/Hospital_Revenue_Factors.parquet') );

In [ ]:
CREATE MULTISET TABLE Hospital_Revenue_Factors AS (
 
select * from Antiselect (
on Hospital_Revenue_Factors_F
USING
Exclude ('Location'
           )
) as dt) WITH DATA;

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3. Creat original_cluster table from S3 Buckets</b></p>

In [2]:
CREATE FOREIGN TABLE
original_cluster_F
USING ( LOCATION('/gs/storage.googleapis.com/solomon_demo_data/CSVDATA/original_cluster.parquet') );

Success: 0 rows affected

In [3]:
CREATE MULTISET TABLE original_cluster AS (
 
select * from Antiselect (
on original_cluster_F
USING
Exclude ('Location'
           )
) as dt) WITH DATA;

Success: 0 rows affected

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3. Creat feature table from S3 Buckets</b></p>

In [4]:
CREATE FOREIGN TABLE
features_F
USING ( LOCATION('/gs/storage.googleapis.com/solomon_demo_data/CSVDATA/features.parquet') );

Success: 0 rows affected

In [5]:
CREATE MULTISET TABLE features AS (
 
select * from Antiselect (
on features_F
USING
Exclude ('Location'
           )
) as dt) WITH DATA;

Success: 0 rows affected

In [ ]:
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5. Check if the  table from S3 Buckets</b></p>

In [26]:
SELECT
    *
FROM
    DBC.Tables
WHERE
      databasename = 'demo_user'
    order by 1;

,DatabaseName,TableName,Version,TableKind,ProtectionType,JournalFlag,CreatorName,RequestText,CommentString,ParentCount,ChildCount,NamedTblCheckCount,UnnamedTblCheckExist,PrimaryKeyIndexId,RepStatus,CreateTimeStamp,LastAlterName,LastAlterTimeStamp,RequestTxtOverflow,AccessCount,LastAccessTimeStamp,UtilVersion,QueueFlag,CommitOpt,TransLog,CheckOpt,TemporalProperty,ResolvedCurrent_Date,ResolvedCurrent_Timestamp,SystemDefinedJI,VTQualifier,TTQualifier
1,demo_user,MyAuthObj,1,X,F,N,demo_user,CREATE AUTHORIZATION MyAuthObj USER '' PASSWORD '',,0,0,0,N,0,,2025-03-12 12:05:50,demo_user,2025-03-12 12:05:50,,,,,N,N,Y,N,N,,,N,,
2,demo_user,final_tab,1,V,F,NN,demo_user,"--Optional query to Join Original Clusters with Teradata create view final_tab as ( SELECT DISTINCT hc.td_clusterid_kmeans AS Teradata_Cluster, original_cluster.cluster_id as Original_Cluster, hf.entity_id, -- Include any other non-aggregated columns you want hf.entity_name, hf.count_admissions, hf.count_beds, hf.count_fte, hf.sum_net_revenue, hf.sum_opex, hf.score_case_mix, hf.special_facility, hf.Exclude_Facility FROM features hf JOIN clusters hc ON hf.entity_id = hc.entity_id LEFT JOIN original_cluster ON hf.entity_id = original_cluster.entity_id );",,0,0,0,N,,,2025-03-19 15:38:10,demo_user,2025-03-19 15:38:10,,,,,N,N,Y,N,N,,,N,,
3,demo_user,original_cluster,1,O,F,NN,demo_user,CREATE MULTISET TABLE original_cluster AS ( select * from Antiselect ( on original_cluster_F USING Exclude ('Location' ) ) as dt) WITH DATA;,,0,0,0,N,,,2025-03-19 15:12:37,demo_user,2025-03-19 15:12:37,,,,1,N,N,Y,Y,N,,,N,,
4,demo_user,KMeans_Model,1,O,F,NN,demo_user,"--perform KMeans Clustering Select * from TD_KMeans ( ON v_features_scaled as InputTable OUT TABLE ModelTable(KMeans_Model) USING IdColumn('entity_id') TargetColumns( 'count_fte' , 'count_beds' ,'count_admissions' ,'score_case_mix','sum_net_revenue' ,'sum_opex') InitialCentroidsMethod('kmeans++') StopThreshold(0.001) NumClusters(6) --Seed(0) MaxIterNum(100) )as dt;",,0,0,0,N,,,2025-03-19 15:32:44,demo_user,2025-03-19 15:32:44,,,,1,N,N,Y,Y,N,,,N,,
5,demo_user,AllStats_unpivoted,1,O,F,NN,demo_user,"CREATE MULTISET TABLE AllStats_unpivoted AS ( SELECT * FROM TD_UnivariateStatistics ( ON Hospital_Revenue_Factors1 AS InputTable USING TargetColumns('Less_Allowances_for_Uncollectible_Notes_Accounts_Receivable', 'Medicaid_Charges', 'Net_Revenue_from_Medicaid', 'Cost_To_Charge_Ratio', 'Outpatient_Total_Charges', 'Less_Contractual_Allowance_and_Discounts_on_Patients_Accounts', 'Total_Other_Income', 'Total_Income', 'Net_Income', 'Net_Income_from_Service_to_Patients') STATS( 'MEAN', 'MEDIAN', 'MODE', 'ZERO VALUES COUNT', 'PERCENTILES', 'MINIMUM', 'MAXIMUM' ) ) as dt) with data;",,0,0,0,N,,,2025-03-19 15:41:24,demo_user,2025-03-19 15:41:24,,,,1,N,N,Y,Y,N,,,N,,
6,demo_user,get_data,1,P,F,N,DBC,,,0,2,0,N,1,,2025-03-09 14:56:42,DBC,2025-03-09 14:56:42,,,,,N,N,Y,N,N,,,N,,
7,demo_user,features,1,O,F,NN,demo_user,CREATE MULTISET TABLE features AS ( select * from Antiselect ( on features_F USING Exclude ('Location' ) ) as dt) WITH DATA;,,0,0,0,N,,,2025-03-19 15:14:15,demo_user,2025-03-19 15:14:15,,,,1,N,N,Y,Y,N,,,N,,
8,demo_user,clusters,1,V,F,NN,demo_user,"create view clusters as SELECT entity_id, td_clusterid_kmeans, CURRENT_DATE() as test FROM TD_KMeansPredict ( ON v_features_scaled AS InputTable ON KMeans_Model as ModelTable DIMENSION USING OutputDistance('false') ) as dt;",,0,0,0,N,,,2025-03-19 15:34:20,demo_user,2025-03-19 15:34:20,,,,,N,N,Y,N,N,,,N,,
9,demo_user,hospital_finance_transformed,1,O,F,NN,demo_user,"CREATE MULTISET TABLE hospital_finance_transformed AS ( SELECT Hospital_Name, County, Less_Allowances_for_Uncollectible_Notes_Accounts_Receivable, Medicaid_Charges, Net_Revenue_from_Medicaid, Cost_To_Charge_Ratio, Outpatient_Total_Charges, Less_Contractual_Allowance_and_Discounts_on_Patients_Accounts, Total_Other_Income, Total_Income, Net_Income, Net_Income_from_Service_to_Patients FROM TD_ColumnTransformer( ON Hospital_Revenue_Factors AS In